In [2]:
from pydantic import BaseModel, Field, model_validator
from typing import Optional

import numpy as np

In [3]:
qubit_id = int

In [4]:
class SimplePhysicalBaseBand(BaseModel):
    frequency: float = Field(ge=0.0)
    if_frequency: Optional[float] = Field(ge=0.0, default=250e6)

class SimplePhysicalChannel(BaseModel):
    sample_time: float = Field(ge=0.0)
    block_size: Optional[int] = Field(ge=1, default=1)
    phase_iq_offset: float = 0.0
    bias: float = 1.0
    acquire_allowed: bool = False
    min_frequency: float = Field(ge=0.0, default=0.0)
    max_frequency: float = np.inf
    baseband: SimplePhysicalBaseBand

In [28]:
class SimplePulseChannel(BaseModel):
    frequency: float = Field(ge=0.0, default=0.0)
    bias: complex = 0.0 + 0.0j
    scale: complex = 1.0 + 0.0j
    fixed_if: bool = False

class MeasurePulseChannel(SimplePulseChannel):
    pass

class DrivePulseChannel(SimplePulseChannel):
    pass

class AcquirePulseChannel(SimplePulseChannel):
    pass

class SecondStatePulseChannel(SimplePulseChannel):
    pass

class AcquirePulseChannel(SimplePulseChannel):
    pass

class CrossResonancePulseChannel(SimplePulseChannel):
    target_qubit : qubit_id

class CrossResonanceCancellationPulseChannel(SimplePulseChannel):
    target_qubit : qubit_id

In [29]:
class ResonatorPulseChannels(BaseModel):
    measure : MeasurePulseChannel
    acquire: AcquirePulseChannel

In [21]:
class QubitPulseChannels(BaseModel):
    drive : DrivePulseChannel

In [22]:
class ArbSeqQubitPulseChannels(QubitPulseChannels):
    second_state : SecondStatePulseChannel
    freq_shift: SimplePulseChannel # What is this?
    macq: SimplePulseChannel # What is this?
    cross_resonance_channels: tuple[CrossResonancePulseChannel]
    cross_resonance_cancellation: tuple[CrossResonanceCancellationPulseChannel]


In [24]:
class SimpleQubit(BaseModel):
    physical_channel: SimplePhysicalChannel
    pulse_channels: QubitPulseChannels
    resonator: ResonatorPulseChannels

In [25]:
class ArbSeqQubit(SimpleQubit):
    physical_channel: SimplePhysicalChannel
    pulse_channels: ArbSeqQubitPulseChannels

In [26]:
class SimpleHardwareModel(BaseModel):
    qubits : dict[qubit_id,SimpleQubit]
    topology : dict[qubit_id, tuple[qubit_id,qubit_id]]

    @model_validator(mode='after')
    def validate_model(self):
        # Check the qubit numbers match
        assert set(self.topology) == set(self.qubits)

In [27]:
class ArbSeqHardwareModel(BaseModel):
    qubits : dict[qubit_id,ArbSeqQubit]
    topology : dict[qubit_id, tuple[qubit_id,qubit_id]]
    # We should validate the qubits match the topology
    # I.e. the set of qubit_ids is the same and the 

    @model_validator(mode='after')
    def validate_couplings(self):
        # Check the qubit topology matches the cross_resonance channels
        pass